In [1]:
key = !op read "op://private/sfm openai api key/password"
assert len(key) == 1
key = key[0]
assert len(key) == 51

In [2]:
import openai
import os
import json
from pprint import pprint
from pydantic import BaseModel

openai.api_key = key

In [51]:
class RunTest(BaseModel):
    include_visual_observation: bool
    agent_test_code: str

In [52]:
def read_test():
    path = r"D:\Repos\Minecraft\Forge\SuperFactoryManager\src\gametest\java\ca\teamdman\sfm\ai\TestChambers.java"
    with open(path, "r") as f:
        return f.read()

In [63]:
def write_test(content: str):
    path = r"D:\Repos\Minecraft\Forge\SuperFactoryManager\src\gametest\java\ca\teamdman\sfm\ai\TestChambers.java"
    with open(path, "w") as f:
        f.write(content)

In [53]:
def indent_text(text, spaces=4):
    space_str = " " * spaces
    return "\n".join(f"{space_str}{line}" for line in text.split("\n"))
print(indent_text("a\nb\nc",8))

        a
        b
        c


In [54]:
def with_agent_content(test: str, new_content: str) -> str:
    """
    We want to remove any previous attempts at the test

    // begin agent code
    item.setPos(Vec3.atCenterOf(helper.absolutePos(pressurePlatePos).offset(0,3,0)));
    // end agent code

    should remove the content between the two comments
    """
    region_start = "        // begin agent code"
    region_end = "        // end agent code"
    start = test.find(region_start)
    end = test.find(region_end)
    return test[:start+len(region_start)] + "\n" + new_content + ("\n" if new_content != "" else "") + test[end:]

In [151]:
def get_agent_content(test: str) -> str:
    region_start = "        // begin agent code"
    region_end = "        // end agent code"
    start = test.find(region_start)
    end = test.find(region_end)
    return test[start+len(region_start)+1:end]

# begin hotkey_test

In [69]:
import pyautogui
import pygetwindow as gw
from time import sleep
import pyperclip

In [70]:
def focus_intellij():
    try:
        title = next(x for x in gw.getAllTitles() if "TestChambers.java" in x)
        windows = gw.getWindowsWithTitle(title)
        assert len(windows) > 0, f"Window not found: {title}"
        windows[0].activate()
    except Exception as e:
        print(f"Error focusing window: {e}")

In [71]:
def is_building():
    return pyautogui.pixel(1947, 622) == (95, 173, 101)

In [167]:
def get_build_output():
    # store cursor position
    x, y = pyautogui.position()
    # click and drag from 2890, 638 to 2895, 638 to select text without activating links
    pyautogui.moveTo(2890, 638)
    pyautogui.mouseDown()
    pyautogui.moveTo(2895, 638)
    pyautogui.mouseUp()
    pyautogui.hotkey('ctrl', 'a')
    pyautogui.hotkey('ctrl', 'c')
    # restore cursor position
    pyautogui.moveTo(x, y)
    return pyperclip.paste().replace('\r', '')

In [73]:
def is_happy_build_output(output):
    happy = """
Executing pre-compile tasks…
Running 'before' tasks
Checking sources
Running 'after' tasks
Finished, saving caches…
Executing post-compile tasks…
Finished, saving caches…
    """.strip()
    return output.strip() == happy

In [141]:
def hot_reload():
    print("[Hot Reload] Focusing IntelliJ")
    focus_intellij()
    print("[Hot Reload] Trigger hot reload")
    pyautogui.hotkey('ctrl', 'alt', 'num0')
    print("[Hot Reload] Wait for build to finish")
    while is_building():
        sleep(0.1)
    sleep(0.5)
    print("[Hot Reload] Copy build output")
    output = get_build_output()
    success = is_happy_build_output(output)
    print(f"[Hot Reload] Build output succeeded: {success}")
    return success, output

# end hotkey_test

In [157]:
messages = [
    {"role": "system", "content": "The assistant is tasked with solving puzzles in a Minecraft game test environment, similar to the video game _Portal_. The assistant is presented a game test with some code at the beginning and end of the test that the agent can not change. The agent is responsible for replacing the code in the middle of the test to cause the test to succeed."},
    {"role": "user", "content": f"Current test content:\n```java\n{with_agent_content(read_test(), '')}```"},
]

In [158]:
def save_convo(messages):
    content = ""
    for msg in messages:
        content += f"# ~=~ {msg['role']}\n{msg['content']}\n"
        if "function_call" in msg:
            content += f"---\n{msg['function_call']}\n"
    with open("convo.md", "w") as f:
        f.write(content)

In [159]:
save_convo(messages)

In [160]:
response0 = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages,
    functions=[
        {
          "name": "run_test",
          "description": "Run the test with the agent code block substituted for the provided content. Requesting a visual observation will slow down the process through a dependency on humans.",
          "parameters": RunTest.model_json_schema()
        },
    ],
    function_call={"name": "run_test"}
)
print(response0)

{
  "id": "chatcmpl-7x81F3RHOsjz9N2KAYNe58e8jsjSL",
  "object": "chat.completion",
  "created": 1694326533,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "run_test",
          "arguments": "{\n  \"include_visual_observation\": false,\n  \"agent_test_code\": \"helper.spawnItem(item);\"\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1750,
    "completion_tokens": 24,
    "total_tokens": 1774
  }
}


In [179]:
run = RunTest(**json.loads(response0.choices[0]["message"]["function_call"]["arguments"]))
write_test(with_agent_content(read_test(), indent_text(run.agent_test_code, 12)))
success, build_output = hot_reload()
if not success:
    print(build_output)

[Hot Reload] Focusing IntelliJ
[Hot Reload] Trigger hot reload
[Hot Reload] Wait for build to finish
[Hot Reload] Copy build output
[Hot Reload] Build output succeeded: False
D:\Repos\Minecraft\Forge\SuperFactoryManager\src\gametest\java\ca\teamdman\sfm\ai\TestChambers.java:103: error: no suitable method found for spawnItem(ItemEntity)
            helper.spawnItem(item);
                  ^
    method GameTestHelper.spawnItem(Item,float,float,float) is not applicable
      (actual and formal argument lists differ in length)
    method GameTestHelper.spawnItem(Item,BlockPos) is not applicable
      (actual and formal argument lists differ in length)


# begin runner_comms

In [169]:
from pathlib import Path
msgdir = Path("./messages")
msgdir.mkdir(parents=True, exist_ok=True)

In [170]:
def run_test():
    # touch messages/run.txt
    (msgdir / "run.txt").touch()

    from time import sleep
    # wait for the file to have test results appended
    while not (msgdir / "run.txt").stat().st_size > 0:
        sleep(0.1)
    
    # read the file
    with open(msgdir / "run.txt", "r") as f:
        results = f.read()

    archive_content = get_agent_content(read_test()) + "\n\n" + results
    
    # clean up the file by renaming it with the time
    from datetime import datetime
    now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    (msgdir / f"run_{now}.txt").write_text(archive_content)
    (msgdir / "run.txt").unlink()

    return results

# end runner_comms

In [172]:
if success:
    test_output = run_test()

In [173]:
fn0 = {
    "role": "function",
    "name": "run_test",
    "content": build_output if not success else test_output,
}
print(fn0)

{'role': 'function', 'name': 'run_test', 'content': 'D:\\Repos\\Minecraft\\Forge\\SuperFactoryManager\\src\\gametest\\java\\ca\\teamdman\\sfm\\ai\\TestChambers.java:103: error: no suitable method found for spawnItem(ItemEntity)\n            helper.spawnItem(item);\n                  ^\n    method GameTestHelper.spawnItem(Item,float,float,float) is not applicable\n      (actual and formal argument lists differ in length)\n    method GameTestHelper.spawnItem(Item,BlockPos) is not applicable\n      (actual and formal argument lists differ in length)'}


In [165]:
response1 = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        *messages,
        response0.choices[0]["message"],
        fn0,
    ],
    functions=[
        {
          "name": "run_test",
          "description": "Run the test with the agent code block substituted for the provided content. Requesting a visual observation will slow down the process through a dependency on humans.",
          "parameters": RunTest.model_json_schema()
        },
    ],
    function_call={"name": "run_test"}
)
print(response1)

{
  "id": "chatcmpl-7x82ajsu8nb2FtuOI86K0g1ySR3ku",
  "object": "chat.completion",
  "created": 1694326616,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "run_test",
          "arguments": "{\n  \"include_visual_observation\": false,\n  \"agent_test_code\": \"helper.spawnItem(Items.DIAMOND, pressurePlatePos);\"\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1900,
    "completion_tokens": 32,
    "total_tokens": 1932
  }
}


In [180]:
if response1.choices[0]["message"]["content"] is not None:
    print(response1.choices[0]["message"]["content"])
if "function_call" in response1.choices[0]["message"]:
    run = RunTest(**json.loads(response1.choices[0]["message"]["function_call"]["arguments"]))
    write_test(with_agent_content(read_test(), indent_text(run.agent_test_code, 12)))
    success, build_output = hot_reload()
    if not success:
        print(build_output)

[Hot Reload] Focusing IntelliJ
[Hot Reload] Trigger hot reload
[Hot Reload] Wait for build to finish
[Hot Reload] Copy build output
[Hot Reload] Build output succeeded: True


In [181]:
if success:
    test_output = run_test()

In [176]:
fn1 = {
    "role": "function",
    "name": "run_test",
    "content": output if not success else test_output,
}

In [178]:
response2 = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        *messages,
        response0.choices[0]["message"],
        fn0,
        response1.choices[0]["message"],
        fn1
    ],
    functions=[
        {
          "name": "run_test",
          "description": "Run the test with the agent code block substituted for the provided content. Requesting a visual observation will slow down the process through a dependency on humans.",
          "parameters": RunTest.model_json_schema()
        },
    ],
)
print(response2)

{
  "id": "chatcmpl-7x85jzVFL1jFDB0TG6fhHLK1OwaD6",
  "object": "chat.completion",
  "created": 1694326811,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The test has passed successfully. The code that I've added to make the test case pass is:\n\n```java\nhelper.spawnItem(Items.DIAMOND, pressurePlatePos);\n```\n\nThe code spawns a diamond item on the position of the pressure plate which, in turn, activates the redstone wire and opens the door."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1948,
    "completion_tokens": 67,
    "total_tokens": 2015
  }
}


In [182]:
save_convo([
    *messages,
    response0.choices[0]["message"],
    fn0,
    response1.choices[0]["message"],
    fn1,
    response2.choices[0]["message"],
])